## Mesh manipulation notebook

In this notebook, you'll import a mesh of your choice, say STL, and reduce the number of faces. Additionally, panel method code `NeumannKelvin.jl` will be run on the mesh with reduced faces.

Before running, first install [https://www.anaconda.com/docs/getting-started/miniconda/install#verify-your-install](miniconda) and then create the python environment using `environment.yml`

Follow these steps in a terminal:
```bash
# Create conda environment
# Run in terminal:
conda env create -f environment.yml
conda activate mesh_simplification
```

Once the environment is set up, select the python kernel `mesh_simplification` within VS Code.

In [1]:
import numpy as np
import trimesh
import sys
sys.path.append('./line-quadric-simplification')
from utils.lineQEM import lineQEM

### Load STL file
Note: `trimesh` supports more geometry formats like `OBJ`, `PLY`: https://trimesh.org/formats.html

In [2]:
# surface_mesh_path = "./meshes/roblox_logo.obj" # roblox logo
surface_mesh_path = "./meshes/LowPolyDolphin.stl" # dolfin
original_mesh = trimesh.load(surface_mesh_path, process=True)
vertices_org = original_mesh.vertices
faces_org = original_mesh.faces
print(f"Original Mesh: {len(faces_org)} faces, {len(vertices_org)} vertices")

Original Mesh: 1456 faces, 730 vertices


### Run the mesh simplification

Mesh simplification is based on edge collapse and uses line quadric error metric (lineQEM) to rank the collapses. Source code is at [lineQEM-GitHub](https://github.com/HTDerekLiu/line-quadric-simplification)

In [3]:
# Specify the target faces
target_fraction = 0.1
num_target_faces = int(faces_org.shape[0] * target_fraction)

# Run the mesh simplification algorithm
vertices_simplified, faces_simplified = lineQEM(vertices_org, faces_org, num_target_faces,
                line_quadric_weight=1e-4, # Lower value -> More uniform faces
                boundary_quadric_weight=100.0, # Preserves the boundary edges i.e edges with only one associated face
                quadric_scalings=np.ones(vertices_org.shape[0]))

# Load original to get face count for printing
print(f"\nSimplified Mesh: {len(faces_simplified)} faces, {len(vertices_simplified)} vertices")

decimation progress: 1000 / 1311

Simplified Mesh: 144 faces, 74 vertices


### Save the simplified mesh

In [4]:
# Save simplified mesh using trimesh (process=True fixes winding order / normals)
from pathlib import Path
simplified_mesh = trimesh.Trimesh(vertices=vertices_simplified, faces=faces_simplified, process=True)
simplified_mesh.fix_normals()  # ensure consistent winding to avoid holes

stem = Path(surface_mesh_path).stem
ext = Path(surface_mesh_path).suffix
output_file = str(Path(surface_mesh_path).parent / f"{stem}_simplified{ext}")
simplified_mesh.export(output_file)  # auto-detects format from extension
print(f"Saved to {output_file}")

Saved to meshes/LowPolyDolphin_simplified.stl


### Run the panel method on the simplified mesh

In [5]:
# Ensure the simplified mesh is saved as STL for Julia's MeshIO
print(f"STL for panel method: {output_file}")

STL for panel method: meshes/LowPolyDolphin_simplified.stl


In [6]:
# pip install juliacall  # uncomment if not installed
from juliacall import Main as jl

# Install required Julia packages (only needed once)
jl.seval("""
    import Pkg
    Pkg.add(["NeumannKelvin", "GeometryBasics", "MeshIO", "FileIO", "StaticArrays"])
""")
print("Julia packages installed.")

[juliapkg] Found dependencies: /Users/sankalpjena/miniconda3/envs/mesh_simplification/lib/python3.10/site-packages/juliapkg/juliapkg.json
[juliapkg] Found dependencies: /Users/sankalpjena/miniconda3/envs/mesh_simplification/lib/python3.10/site-packages/juliacall/juliapkg.json
[juliapkg] Locating Julia ^1.10.3
[juliapkg] Using Julia 1.12.5 at /Users/sankalpjena/.julia/juliaup/julia-1.12.5+0.aarch64.apple.darwin14/Julia-1.12.app/Contents/Resources/julia/bin/julia
[juliapkg] Using Julia project at /Users/sankalpjena/miniconda3/envs/mesh_simplification/julia_env
[juliapkg] Writing Project.toml:
           | [deps]
           | PythonCall = "6099a3de-0909-46bc-b1f4-468b9a2dfc0d"
           | OpenSSL_jll = "458c3c95-2e84-50aa-8efc-19380b2a3a95"
           | 
           | [compat]
           | PythonCall = "=0.9.31"
           | OpenSSL_jll = "3.0.0 - 3.6"
[juliapkg] Installing packages:
           | import Pkg
           | Pkg.Registry.update()
           | Pkg.add([
           |   Pkg.Packa

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
    Updating `~/miniconda3/envs/mesh_simplification/julia_env/Project.toml`
  [6099a3de] + PythonCall v0.9.31
  [458c3c95] ~ OpenSSL_jll ⇒ v3.5.4+0
    Updating `~/miniconda3/envs/mesh_simplification/julia_env/Manifest.toml`
  [992eb4ea] + CondaPkg v0.2.34
  [9a962f9c] + DataAPI v1.16.0
  [e2d170a0] + DataValueInterfaces v1.0.0
  [82899510] + IteratorInterfaceExtensions v1.0.0
  [692b3bcd] + JLLWrappers v1.7.1
  [682c06a0] + JSON v1.4.0
  [1914dd2f] + MacroTools v0.5.16
  [0b3b1443] + MicroMamba v0.1.15
  [bac558e1] + OrderedCollections v1.8.1
  [69de0a69] + Parsers v2.8.3
  [fa939f87] + Pidfile v1.3.0
  [aea7be01] + PrecompileTools v1.3.3
  [21216c6a] + Preferences v1.5.1
  [6099a3de] + PythonCall v0.9.31
  [6c6a2e73] + Scratch v1.3.0
  [ec057cc2] + StructUtils v2.6.3
  [3783bdb8] + TableTraits v1.0.1
  [bd369af6] + Tables v1.12.1
  [e17b2a0c] + UnsafePointers v1.0.0
  [f8abcde7] + micromamba_

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


   Resolving package versions...
    Updating `~/miniconda3/envs/mesh_simplification/julia_env/Project.toml`
  [5789e2e9] + FileIO v1.18.0
  [5c1252a2] + GeometryBasics v0.5.10
  [7269a6da] + MeshIO v0.5.3
  [7f078b06] + NeumannKelvin v0.9.2
  [90137ffa] + StaticArrays v1.9.16
    Updating `~/miniconda3/envs/mesh_simplification/julia_env/Manifest.toml`
  [621f4979] + AbstractFFTs v1.5.0
  [6a4ca0a5] + AcceleratedKernels v0.4.3
  [7d9f7c33] + Accessors v0.1.43
  [79e6a3ab] + Adapt v4.4.0
  [dce04be8] + ArgCheck v2.5.0
  [a9b6321e] + Atomix v1.1.2
  [d360d2e6] + ChainRulesCore v1.26.0
  [3da002f7] + ColorTypes v0.12.1
  [861a8166] + Combinatorics v1.1.0
  [38540f10] + CommonSolve v0.2.6
  [bbf7d656] + CommonSubexpressions v0.3.1
  [34da2185] + Compat v4.18.1
  [a33af91c] + CompositionsBase v0.1.2
  [187b0558] + ConstructionBase v1.6.0
  [a8cc5b0e] + Crayons v4.1.1
  [82cc6244] + DataInterpolations v8.9.0
  [864edb3b] + DataStructures v0.19.3
  [85a47980] + Dictionaries v0.4.6
  [163ba53b

Julia packages installed.


In [7]:
# Load Julia packages
jl.seval("""
    using NeumannKelvin, GeometryBasics, MeshIO, FileIO, StaticArrays
""")
print("Julia packages loaded.")

Julia packages loaded.


In [8]:
# 1. Set-up: Load STL and panelize
jl.stl_path = output_file  # pass Python path to Julia
jl.seval("""
    panels = load(stl_path) |> panelize
    println("Panelized: $(length(panels)) panels")
""")

# 2. Solve: Create panel system and solve
jl.seval("""
    sys = BodyPanelSystem(panels, U=SA[1,0,0], wrap=PanelTree)
    gmressolve!(sys, verbose=true)
""")

# 3. Retrieve results back into Python
Cp = jl.seval("collect(cₚ(sys))")
print(f"\nPanel method complete. Cp range: [{min(Cp):.4f}, {max(Cp):.4f}]")

Panelized: 144 panels
SimpleStats
 niter: 14
 solved: true
 inconsistent: false
 indefinite: false
 npcCount: 0
 residuals: []
 Aresiduals: []
 κ₂(A): []
 timer: 135.23ms
 status: solution good enough given atol and rtol


Panel method complete. Cp range: [-27.4240, 0.9914]
